# NCI GDC Data Analysis

Datasets were pulled from https://portal.gdc.cancer.gov/analysis_page?app=CohortBuilder&tab=general_diagnosis 
They were filtered for Breast Cancer before pulled. Need to check with Arjita on the exact process of that

## Load Packages

In [4]:
import pandas as pd
import plotly.express as px
import numpy as np
import requests
from io import StringIO
import seaborn as sns
import matplotlib.pyplot as plt


pd.set_option('future.no_silent_downcasting', True)

## Import Data from Github

Pull in gene dataset. 

In [ ]:
response = requests.get('https://raw.githubusercontent.com/aaditya0106/cancer-dashboard/main/Data/HierCluster.2024-11-19.tsv')
if response.status_code == 200:
    gene = pd.read_csv(StringIO(response.text), sep='\t')
    print("Data loaded successfully!")
else:
    print(f"Failed to fetch data: {response.status_code}")

Pull in clinical dataset.

In [ ]:
response = requests.get('https://raw.githubusercontent.com/aaditya0106/cancer-dashboard/main/Data/clinical.tsv')
if response.status_code == 200:
    clinical = pd.read_csv(StringIO(response.text), sep='\t')
    print("Data loaded successfully!")
else:
    print(f"Failed to fetch data: {response.status_code}")

## Basic gene data exploration

In [ ]:
gene.head()

In [ ]:
gene.shape

In [ ]:
gene.dtypes

Determine if any values are Null. There are no null values and there are no duplicate samples. 

In [ ]:
gene.isnull().any().value_counts()

In [ ]:
gene['Case'].duplicated().sum()

## Basic clinical data exploration

In [ ]:
clinical.head()

In [ ]:
clinical.columns

In [ ]:
clinical.shape

Make sure tissue of origin is breast related. These are the types of tissues currently included. 

In [ ]:
# Generate the summary table of what tissue_or_organ_of_origin is
summary_table = clinical['tissue_or_organ_of_origin'].value_counts()

# Display the summary table
print(summary_table)

Remove rows where tissue or organ of origin are Not Reported or '--

In [ ]:
clinicalBreast = clinical[~clinical['tissue_or_organ_of_origin'].isin(['Not Reported', "'--"])]

# Check the result
print(clinicalBreast['tissue_or_organ_of_origin'].value_counts())


In [ ]:
clinicalBreast.head()

Check that 73 invalid rows were removed

In [ ]:
clinicalBreast.shape

Get number of duplicate case_submitter_id numbers. These are patients with multiple rows of data.

In [ ]:
clinicalBreast['case_submitter_id'].duplicated().sum()

Get a table of the duplicates

In [ ]:
# Identify all rows with duplicates
all_duplicates = clinicalBreast[clinicalBreast['case_submitter_id'].duplicated(keep=False)]

## Merge gene and clinicalBreast

In [ ]:
geneClinical = gene.merge(clinicalBreast, left_on='Case', right_on='case_submitter_id', how='inner')

In [ ]:
geneClinical.shape

Check results for a duplicate case_submitter_id. We want to make sure each case_submitter_id/case always gets the same gene expression data

In [ ]:
geneClinical['case_submitter_id'].duplicated().sum()

Get a table of the duplicates

In [ ]:
# Identify all rows with duplicates
all_duplicatesMerge = geneClinical[geneClinical['case_submitter_id'].duplicated(keep=False)]

In [ ]:
all_duplicatesMerge.head()

If I drop clinical columns, then drop duplicates. Do I get a table 1000 rows long?

In [ ]:
short = geneClinical.iloc[:, 0:1001]
short.shape

In [ ]:
short.columns

In [ ]:
shortNoDup = short.drop_duplicates()
shortNoDup.shape

I have less than the 1000 samples I started with. Check what happened to one sample. 

In [ ]:
# Get indices where 'Case' in 'gene' is not in 'shortNoDup'
indices_not_in_shortNoDup = gene[~gene['Case'].isin(shortNoDup['Case'])].index

# Print the result
print(indices_not_in_shortNoDup)


In [ ]:
# print rows that do not have a case in the merged dataset
gene.iloc[indices_not_in_shortNoDup, :]

In [ ]:
gene[gene['Case'] == 'TCGA-BH-A0B2']

In [ ]:
clinicalBreast[clinicalBreast['case_submitter_id'] == 'TCGA-BH-A0B2']

In [ ]:
clinical[clinical['case_submitter_id'] == 'TCGA-BH-A0B2']

Case 'TCGA-BH-A0B2' was in clinical. However, the tissue or ogan was equal to '-- so it got removed when I created breastClinical. It is rightfully not present in the merged data

## Explore the merged data

In [ ]:
geneClinical2 = geneClinical.replace("'--", np.NaN)

In [ ]:
# Report the number of nulls per column
null_counts = geneClinical2.isnull().sum()


In [ ]:
# Filter columns where the number of nulls is <= 1800 (Total number of rows is 1854)
columns_to_keep = null_counts[null_counts <= 1800].index

# Select only the columns to keep
filtered_geneClinical = geneClinical2[columns_to_keep]

print(f"Columns removed: {set(geneClinical2.columns) - set(filtered_geneClinical.columns)}")
print(f"Filtered DataFrame shape: {filtered_geneClinical.shape}")


There are 1854 rows. When we removed columns that had >1800 nulls then we removed 184 columns. Now checking if there are other columns that have a high percentage of nulls

In [ ]:
filtered_geneClinical.isnull().sum().sort_values(ascending = False)

I am going to remove year_of_death and days_to_death as these are at best 87% null. After those that amount of nulls drops sharply so I will keep those clinical columns

In [ ]:
# Drop the specified columns from the DataFrame
filtered_geneClinical2 = filtered_geneClinical.drop(['year_of_death', 'days_to_death'], axis=1)

# Display the updated DataFrame shape
print(filtered_geneClinical2.shape)


## Normalize gene expresion values.

Normalize gene expression values. 

In [ ]:
# select only the columns with gene expression data
df = filtered_geneClinical2.iloc[:, 1:1001]
df.head()

In [ ]:
df

Check how the values within expression are distributed. It appears they have already had a zscale normalization

In [ ]:
df.describe()

Concatenate 30 genes with clinical data. Reduce down to 100 samples

### Selected genes related to breast cancer from the following papers. Then filter to only include these columns
https://pmc.ncbi.nlm.nih.gov/articles/PMC6147049/
https://pmc.ncbi.nlm.nih.gov/articles/PMC9299843/

In [ ]:
# estrogen receptor ESR1 and ESR2
# List comprehension to find column names starting with 'ESR'
esr_columns = [col for col in df.columns if col.startswith('ESR')]

# Print the result
print(esr_columns)

# These are epithelial splicing regulatory proteins genes. Not of interest

# HER2 - overexpression can lead to uncontrolled cell growth and development of cancer
# did the same search type. not HER genes

In [ ]:
# no : WNT, ATM, PALB, MYC, DKK1, PTCH-1, GLI-1, SHH, SMOH, PIK3CA, PIK3R1, AKT
# no: PTEN, LKB1, FANCN, OBSCN, ATR, FGFR, FOX, TSC
#TP53INP1 breast paper
# CDH1 (inherited mutations cause hereditary diffuse gastric cancer with an increased risk of invasive lobular breast cancer)
# ALDH1A1, CD44, CCND1, CCND2, TOP2A, DUSP4
# mutations of ERBB2, ERBB3, CDK6
# LAMP5, LTF, MMP9, S100P, TPM1, CCL18, CDCA7, CDKN2D, E2F2, EPCAM, FOS, FOSB, HES6, LAPTM4B mammaprint paper
# UBE2C, UHRF1 mammaprint paper
interest_columns = [col for col in df.columns if col.startswith('LAP')]

# Print the result
print(interest_columns)

In [ ]:
# list of breast cancer related genes found in the dataset
genesOfInterest = [
    "TP53INP1", "CDH1", "ALDH1A1", "CD44", "CCND1", "CCND2", "TOP2A", 
    "DUSP4", "ERBB2", "ERBB3", "CDK6", "LAMP5", "LTF", "MMP9", "S100P", 
    "TPM1", "CCL18", "CDCA7", "CDKN2D", "E2F2", "EPCAM", "FOS", "FOSB", 
    "HES6", "LAPTM4B", "UBE2C", "UHRF1"
]

len(genesOfInterest)

In [ ]:
# Select columns from the dataframe
selected_columns = filtered_geneClinical2[
    ['Case'] + genesOfInterest + filtered_geneClinical2.columns[1002:].tolist()
]

# Display the resulting dataframe
print(selected_columns.head())


In [ ]:
selected_columns.columns

In [ ]:
# commenting out this is when I was selecting first 30 genes
# combined_df = pd.concat([filtered_geneClinical2.iloc[:, 0:31], filtered_geneClinical2.iloc[:, 1002:]], axis=1)

#reduce to 100 samples
combined_df = selected_columns.iloc[:, :]
# Check the combined dataset
print("Combined Dataset:")
print(combined_df.head())


In [ ]:
combined_df.columns

In [ ]:
# List of all columns except the gene expression columns

# Create a list of column names not in genesOfInterest
id_vars = [col for col in combined_df.columns if col not in genesOfInterest]

# Display the list of column names
print(id_vars)


In [ ]:
# Use melt to reshape the DataFrame
reshaped_df = combined_df.melt(id_vars=id_vars, var_name='Gene', value_name='Expression')

# Check the reshaped DataFrame
print(reshaped_df.head())


In [ ]:
reshaped_df_drop = reshaped_df.drop(columns='case_submitter_id')

In [ ]:
reshaped_df_drop.duplicated().sum()

In [ ]:
#reshaped_df_drop.to_csv("geneClinicalClean.csv")

In [ ]:
reshaped_df_drop

### Consolidate the sub-stages within a stage

In [ ]:
reshaped_df_drop['ajcc_pathologic_stage'].value_counts()

In [ ]:
unique_expression = reshaped_df_drop.groupby(['Case', 'Gene'])['Expression'].first().unstack()

In [ ]:
reshaped_df_drop['Cancer Stage'] = reshaped_df_drop['ajcc_pathologic_stage'].apply(
    lambda x: (
        'Stage I' if isinstance(x, str) and ('Stage I' in x and 'Stage IA' in x)
        else 'Stage II' if isinstance(x, str) and ('Stage IIA' in x or 'Stage IIB' in x)
        else 'Stage III' if isinstance(x, str) and ('Stage IIIA' in x or 'Stage IIIB' in x or 'Stage IIIC' in x)
        else 'Stage IV' if isinstance(x, str) and 'Stage IV' in x
        else None  # Handle missing or unexpected values
    )
)

# Display the counts for each stage
stage_counts = reshaped_df_drop['Cancer Stage'].value_counts()
print(stage_counts)



In [ ]:
reshaped_df_drop.to_csv("geneClinicalCleanStageGeneUpdate.csv")

In [ ]:
unique_expression = unique_expression.astype(float).fillna(0)  # To Ensure all values are numeric

In [ ]:
unique_expression

In [ ]:
# # If 'Case' is the index in unique_expression, reset the index first
# if 'Case' not in unique_expression.columns:
#     unique_expression.reset_index(inplace=True)

# # Add the 'Cancer Stage' column to unique_expression by mapping from reshaped_df
# unique_expression['Cancer Stage'] = unique_expression['Case'].map(
#     reshaped_df.set_index('Case')['Cancer Stage']
# )

# # Display the updated unique_expression DataFrame
# print(unique_expression.head())


In [ ]:
# unique_expression.index

In [ ]:
# Use seaborn for a static heatmap as an alternative
plt.figure(figsize=(12, 8))
sns.heatmap(
    unique_expression,
    cmap="viridis",
    cbar_kws={'label': 'Expression Level'},
    xticklabels=True,
    yticklabels=True
)

plt.title("Heatmap of Gene Expression Levels")
plt.xlabel("Genes")
plt.ylabel("Cases")
plt.show()


In [ ]:
unique_expression.reset_index(inplace=True)
unique_expression_melted = unique_expression.melt(id_vars='Case', var_name='Gene', value_name='Expression')

In [ ]:
unique_expression

In [ ]:


# Map the 'Cancer Stage' values from reshaped_df to unique_expression
unique_expression['Cancer Stage'] = unique_expression['Case'].map(
    reshaped_df_drop.drop_duplicates(subset='Case').set_index('Case')['Cancer Stage']
)

# Display the updated unique_expression DataFrame
print(unique_expression.head())


In [ ]:
heatmap = px.density_heatmap(
    unique_expression_melted,
    x="Gene",
    y="Case",
    z="Expression",
    color_continuous_scale="Blues",
    range_color=(-2, 30),
    title="Heatmap of Gene Expression Levels",
    labels={'Expression': 'Expression Level'}
)

heatmap.show()

# heatmap_file_path = 'heatmap.html'
# heatmap.write_html(heatmap_file_path)

In [ ]:
# Add 'Cancer Stage' to unique_expression_melted
unique_expression_melted = unique_expression_melted.merge(
    reshaped_df_drop[['Case', 'Cancer Stage']].drop_duplicates(),
    on='Case',
    how='left'
)

unique_expression_melted
# # Create the heatmap with 'Cancer Stage' in the tooltip
# heatmap = px.density_heatmap(
#     unique_expression_melted,
#     x="Gene",
#     y="Case",
#     z="Expression",
#     color_continuous_scale="Blues",
#     range_color=(-2, 30),
#     title="Heatmap of Gene Expression Levels",
#     labels={'Expression': 'Expression Level'},
#     hover_data={'Cancer Stage': True}  # Include 'Cancer Stage' in the tooltip
# )

# # Display the heatmap
# heatmap.show()


In [ ]:
# Filter out "Unknown" values for AJCC columns
aggregated_data = reshaped_df_drop.groupby(
    ['Case', 'Cancer Stage', 'ajcc_pathologic_n', 'ajcc_pathologic_m', 'ajcc_pathologic_t'], as_index=False
).agg({'Expression': 'mean'})

filtered_data = aggregated_data[reshaped_df_drop['ajcc_pathologic_stage'] != "Unknown"]

sorted_stages = ['Stage I', 'Stage II', 'Stage III']
filtered_data['Cancer Stage'] = pd.Categorical(filtered_data['Cancer Stage'], categories=sorted_stages, ordered=True)

# Count cases by stage
stage_counts = filtered_data['Cancer Stage'].value_counts().sort_index()

# Create a DataFrame for Plotly
plot_data = stage_counts.reset_index()
plot_data.columns = ['Cancer Stage', 'Number of Cases']

fig = px.bar(
    plot_data,
    x='Cancer Stage',
    y='Number of Cases',
    text='Number of Cases',
    title='Distribution of Cases by AJCC Pathologic Stage',
    labels={'Cancer Stage': 'AJCC Pathologic Stage', 'Number of Cases': 'Number of Cases'},
)

fig.update_traces(textposition='outside', marker_color='lightblue', marker_line_color='black', marker_line_width=1)
fig.update_layout(xaxis_title='AJCC Pathologic Stage', yaxis_title='Number of Cases', xaxis_tickangle=45)

fig.show()



In [ ]:
# Filter out "Unknown" values for AJCC Pathologic N stages
filtered_data_state_n = aggregated_data[reshaped_df_drop['ajcc_pathologic_n'] != "Unknown"]

fig = px.box(
    filtered_data_state_n,
    x='ajcc_pathologic_n',
    y='Expression',
    color='ajcc_pathologic_n',  # Optional: Assign a unique color to each category
    title="Average Gene Expression Across AJCC Pathologic N Stages (Aggregated by Case)",
    labels={
        'ajcc_pathologic_n': 'AJCC Pathologic N Stage',
        'Expression': 'Expression Level'
    },
    color_discrete_sequence=px.colors.qualitative.Pastel 
)

fig.update_layout(
    xaxis_title="AJCC Pathologic N Stage",
    yaxis_title="Expression Level",
    xaxis_tickangle=45,
    showlegend=False  
)


fig.show()


In [ ]:
# Filter out "Unknown" values for AJCC Pathologic M stages
filtered_data_state_m = aggregated_data[reshaped_df_drop['ajcc_pathologic_m'] != "Unknown"]

fig = px.box(
    filtered_data_state_m,
    x='ajcc_pathologic_m',
    y='Expression',
    color='ajcc_pathologic_m', 
    title="Average Gene Expression Across AJCC Pathologic M Stages (Aggregated by Case)",
    labels={
        'ajcc_pathologic_m': 'AJCC Pathologic M Stage',
        'Expression': 'Expression Level'
    },
    color_discrete_sequence=px.colors.qualitative.Pastel 
)


fig.update_layout(
    xaxis_title="AJCC Pathologic M Stage",
    yaxis_title="Expression Level",
    xaxis_tickangle=45,
    showlegend=False  # Hide legend if not needed
)

# Show the interactive plot
fig.show()


In [ ]:
# Filter out "Unknown" values for AJCC Pathologic T stages
filtered_data_state_t = aggregated_data[reshaped_df_drop['ajcc_pathologic_t'] != "Unknown"]

fig = px.box(
    filtered_data_state_t,
    x='ajcc_pathologic_t',
    y='Expression',
    color='ajcc_pathologic_t',  
    title="Average Gene Expression Across AJCC Pathologic T Stages (Aggregated by Case)",
    labels={
        'ajcc_pathologic_t': 'AJCC Pathologic T Stage',
        'Expression': 'Expression Level'
    },
    color_discrete_sequence=px.colors.qualitative.Pastel 
)


fig.update_layout(
    xaxis_title="AJCC Pathologic T Stage",
    yaxis_title="Expression Level",
    xaxis_tickangle=45,
    showlegend=False 
)

fig.show()


In [ ]:
# Aggregate data by case (mean expression for cases with multiple rows)
aggregated_data = reshaped_df_drop.groupby(['Case', 'primary_diagnosis'], as_index=False).agg({'Expression': 'mean'})

# Remove "Not Reported" values from the primary diagnosis column
filtered_data = aggregated_data[aggregated_data['primary_diagnosis'] != "Not Reported"]

# Shorten or format long labels with line breaks
filtered_data['primary_diagnosis'] = filtered_data['primary_diagnosis'].apply(
    lambda x: '<br>'.join(x.split(' ', 3)[:3]) if len(x.split(' ')) > 3 else x.replace(' ', '<br>')
)

# Create a violin plot using Plotly
fig = px.violin(
    filtered_data,
    x='primary_diagnosis',
    y='Expression',
    box=True,  
    points=False,
    color='primary_diagnosis',  
    title="Gene Expression Distribution Across Primary Diagnosis Categories (Aggregated by Case)",
    labels={
        'primary_diagnosis': 'Primary Diagnosis',
        'Expression': 'Expression Level'
    },
    color_discrete_sequence=px.colors.qualitative.Set2 
)


fig.update_layout(
    xaxis_title="Primary Diagnosis",
    yaxis_title="Expression Level",
    xaxis_tickangle=0,  
    showlegend=False,  
    width=1200, 
    height=700  
)


fig.show()

